In [1]:
# Import Libraries
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import pickle
import seaborn as sns

import warnings
warnings.filterwarnings('error')

# Headers for different tables
meta_data_header = ['FrameNumber', 'Time', 'Stage_x', 'Stage_y', 'Centroid_x', 'Centroid_y',
                    'Midpoint_x', 'Midpoint_y', 'Head_x', 'Head_y', 'Tail_x', 'Tail_y', 'MouthHook_x', 'MouthHook_y',
                    'LeftMHhook_x', 'LeftMHhook_y', 'RightMHhook_x', 'RightMHhook_y',
                    'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y', 'RightDorsalOrgan_x', 'RightDorsalOrgan_y',
                    'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y', 'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y',
                    'RightBolwigOrgan_x', 'RightBolwigOrgan_y', 'V9_x', 'V9_y', 'V10_x', 'V10_y', 'V11_x', 'V11_y',
                    'V12_x', 'V12_y', 'V13_x', 'V13_y', 'V14_x', 'V14_y', 'V15_x', 'V15_y', 'V16_x', 'V16_y',
                    'MouthHook_votes', 'LeftMHhook_votes', 'RightMHhook_votes', 'LeftDorsalOrgan_votes',
                    'RightDorsalOrgan_votes', 'CenterBolwigOrgan_votes', 'LeftBolwigOrgan_votes', 'RightBolwigOrgan_votes',
                    'V9_votes', 'V10_votes', 'V11_votes', 'V12_votes', 'V13_votes', 'V14_votes', 'V15_votes',
                    'V16_votes', 'Num_Key_points']

coordinate_header = ['FrameNumber', 'MouthHook_x', 'MouthHook_y', 'LeftMHhook_x', 'LeftMHhook_y',
                     'RightMHhook_x', 'RightMHhook_y', 'LeftDorsalOrgan_x', 'LeftDorsalOrgan_y',
                     'RightDorsalOrgan_x', 'RightDorsalOrgan_y', 'CenterBolwigOrgan_x', 'CenterBolwigOrgan_y',
                     'LeftBolwigOrgan_x', 'LeftBolwigOrgan_y', 'RightBolwigOrgan_x', 'RightBolwigOrgan_y']

distance_header = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan', 'LeftBolwigOrgan', 'RightBolwigOrgan']

roc_headers = ['true_positive', 'false_positive', 'true_negative',
           'false_negative', 'positive', 'negative',
           'true_positive_rate', 'false_positive_rate', 'total_negative',
           'total_positive', 'true_negative_rate', 'false_negative_rate',
           'precision', 'recall', 'vote_threshold']

def getROCcurve(allData, detectBodyParts, outlierDistAll, vote_thresholds):
    rocDataAll = {}
    for bp in tqdm_notebook(detectBodyParts, total=len(detectBodyParts), unit="bp", desc='Body parts'):
        rocDataAll[bp] = {}
        for out_dist in tqdm_notebook(outlierDistAll,  total=len(outlierDistAll), unit="thresh", desc='Outliers', leave=False):
            rocDataAll[bp][out_dist] = pd.DataFrame([], columns=roc_headers)
            vt_iter = -1
            for vt in tqdm_notebook(vote_thresholds, total=len(vote_thresholds), unit="vote", desc='Votes', leave=False):
                vt_iter += 1
                true_positive = 0;
                false_positive = 0;
                true_negative = 0;
                false_negative = 0;
                positives = 0;
                negatives = 0;

                ## For each Vote Threshold Find TP, FP etc.
                for row_index, row in allData.iterrows():
                    annotation = row.loc[[bp+'_x_A', bp+'_y_A']].values
                    detection = row.loc[[bp+'_x_T', bp+'_y_T']].values
                    total_kp = row.loc['Num_Key_points'] + 1.0
                    vote = float(row.loc[[bp+'_votes']].values)/float(total_kp)
                    distance = row.loc[[bp+'_dist']].values
                    detection_present = False;
                    annotation_present = False;

                    if (annotation[0] >= 0) and (annotation[1] >= 0):
                        positives += 1;
                        annotation_present = True;
                        if (vote > vt) and (distance <= out_dist):
                            detection_present = True;
                    else:
                        negatives += 1;
                        if (vote > vt):
                            detection_present = True;

                    if annotation_present and detection_present:
                        true_positive += 1;
                    elif annotation_present and (not detection_present):
                        false_negative += 1;
                    elif (not annotation_present) and (not detection_present):
                        true_negative += 1;
                    elif (not annotation_present) and detection_present:
                        false_positive += 1;

            #             print 'P(%d) = TP(%d)+FN(%d)' %(positives, true_positive, false_negative)
            #             print 'N(%d) = TN(%d)+FP(%d)' %(negatives, true_negative, false_positive)

                assert positives == (true_positive+false_negative), sprintf('positives(%d) ~= true_positive(%d)+false_negative(%d)+false_positive(%d)', positives, true_positive, false_negative, false_positive)
                assert negatives == (true_negative+false_positive), sprintf('negatives(%d) == true_negative(%d)+false_positive(%d)', negatives, true_negative, false_positive)

                rocDataAll[bp][out_dist].loc[vt_iter, 'vote_threshold'] = vt;
                rocDataAll[bp][out_dist].loc[vt_iter, 'true_positive'] = true_positive;
                rocDataAll[bp][out_dist].loc[vt_iter, 'false_positive'] = false_positive;
                rocDataAll[bp][out_dist].loc[vt_iter, 'true_negative'] = true_negative;
                rocDataAll[bp][out_dist].loc[vt_iter, 'false_negative'] = false_negative;
                rocDataAll[bp][out_dist].loc[vt_iter, 'positive'] = positives;
                rocDataAll[bp][out_dist].loc[vt_iter, 'total_negative'] = false_positive + true_negative;
                rocDataAll[bp][out_dist].loc[vt_iter, 'total_positive'] = true_positive + false_negative;
                rocDataAll[bp][out_dist].loc[vt_iter, 'negative'] = negatives;
                rocDataAll[bp][out_dist].loc[vt_iter, 'true_positive_rate'] = float(true_positive)/float(positives);

                if negatives > 0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'false_positive_rate'] = float(false_positive)/float(negatives);
                    rocDataAll[bp][out_dist].loc[vt_iter, 'true_negative_rate'] = float(true_negative)/float(negatives);
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'false_positive_rate'] = 0.0;
                    rocDataAll[bp][out_dist].loc[vt_iter, 'true_negative_rate'] = 0.0;

                rocDataAll[bp][out_dist].loc[vt_iter, 'false_negative_rate'] = float(false_negative)/float(positives);

                if (float(true_positive) + float(false_positive)) > 0.0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'precision'] = float(true_positive)/(float(true_positive) + float(false_positive));
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'precision'] = 1

                if (float(true_positive) + float(false_negative)) > 0.0:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'recall'] = float(true_positive)/(float(true_positive) + float(false_negative));
                else:
                    rocDataAll[bp][out_dist].loc[vt_iter, 'recall'] = np.nan

    return rocDataAll

## Plot ROC Curve
def plotROC_curve(rocDat, detectBodyParts, outlierDistAll, figName):
    row, col = (0, 0)
    fig, ax = plt.subplots(2, 3, figsize=(18, 15), dpi=90)

    for bp in detectBodyParts:
        for out_thresh in outlierDistAll:
            fpr = np.array(rocDat[bp][out_thresh].loc[:, 'false_positive_rate'].values, dtype=np.float16)
            tpr = np.array(rocDat[bp][out_thresh].loc[:, 'true_positive_rate'].values, dtype=np.float16)
            ax[row, col].plot(fpr, tpr, linewidth=3, label=int(out_thresh))
        
        ax[row, col].set_xlabel('False Positive Rate', fontsize=18)
        ax[row, col].set_ylabel('True Positive Rate', fontsize=18)
        ax[row, col].set_title(bp, fontsize=20)
        ax[row, col].set_aspect('equal', adjustable='box')
        ax[row, col].set_xlim([-0.05, 1.05])
        ax[row, col].set_ylim([-0.05, 1.05])
        ax[row, col].spines['right'].set_visible(False)
        ax[row, col].spines['top'].set_visible(False)
        ax[row, col].legend(loc = 'lower right', frameon=False, fontsize=14, title='Distance')
        for label in (ax[row, col].get_xticklabels() + ax[row, col].get_yticklabels()):
            label.set_fontsize(18)

        col += 1
        if col > 2:
            row += 1
            col = 0

    plt.savefig(figName, dpi=300)

def plotPR_curve(rocDat, detectBodyParts, outlierDistAll, figName):
    row, col = (0, 0)
    fig, ax = plt.subplots(2, 3, figsize=(18, 15), dpi=90)

    for bp in detectBodyParts:
        for out_thresh in outlierDistAll:
            precision = np.array(rocDat[bp][out_thresh].loc[:, 'precision'].values, dtype=np.float16)
            recall = np.array(rocDat[bp][out_thresh].loc[:, 'recall'].values, dtype=np.float16)
            average_precision = np.sum(np.multiply((recall[1:]-recall[0:-1]), precision[1:]))

            ax[row, col].plot(recall, precision, linewidth=3, label=int(out_thresh))
        
        ax[row, col].set_xlabel('Recall', fontsize=18)
        ax[row, col].set_ylabel('Precision', fontsize=18)
        ax[row, col].set_title('%s'%(bp), fontsize=20)
        ax[row, col].set_aspect('equal', adjustable='box')
        ax[row, col].set_xlim([-0.05, 1.05])
        ax[row, col].set_ylim([-0.05, 1.05])
        ax[row, col].spines['right'].set_visible(False)
        ax[row, col].spines['top'].set_visible(False)
        ax[row, col].legend(loc = 'lower left', frameon=False, fontsize=14, title='Distance')
        for label in (ax[row, col].get_xticklabels() + ax[row, col].get_yticklabels()):
            label.set_fontsize(18)

        col += 1
        if col > 2:
            row += 1
            col = 0

    plt.savefig(figName, dpi=300)

def plotRecall_curve(rocDat, detectBodyParts, outlierDistAll, figName):
    row, col = (0, 0)
    fig, ax = plt.subplots(2, 3, figsize=(18, 15), dpi=90)

    for bp in detectBodyParts:
        for out_thresh in outlierDistAll:
            recall = np.array(rocDat[bp][out_thresh].loc[:, 'recall'].values, dtype=np.float16)
            vote_threshold = np.array(rocDat[bp][out_thresh].loc[:, 'vote_threshold'].values, dtype=np.float16)            
            ax[row, col].plot(vote_threshold, recall, linewidth=3, label=int(out_thresh))
        
        ax[row, col].set_xlabel('Vote Threshold', fontsize=18)
        ax[row, col].set_ylabel('Recall', fontsize=18)
        ax[row, col].set_title('%s'%(bp), fontsize=20)
        ax[row, col].set_aspect('equal', adjustable='box')
        ax[row, col].set_xlim([-0.05, max(vote_threshold) + 0.05])
        ax[row, col].set_ylim([-0.05, 1.05])
        ax[row, col].spines['right'].set_visible(False)
        ax[row, col].spines['top'].set_visible(False)
        ax[row, col].legend(loc = 'lower left', frameon=False, fontsize=14, title='Distance')
        for label in (ax[row, col].get_xticklabels() + ax[row, col].get_yticklabels()):
            label.set_fontsize(18)

        col += 1
        if col > 2:
            row += 1
            col = 0

    plt.savefig(figName, dpi=300)
    
def plotDistanceBoxPlot(meta_coord, fname, test_string_2):
    distance_header_2 = [head+'_dist' for head in distance_header]
    distance_2 = meta_coord.loc[:, distance_header_2].copy()
    distance_2.columns = distance_header
    distance_2.reset_index(inplace=True)
    
    distance_melt = pd.melt(distance_2, id_vars='FrameNumber', value_vars=distance_header[:-2],
                            var_name='BodyPart', value_name='Distance')
    distance_melt = distance_melt[~distance_melt.loc[:, 'Distance'].isnull()]
    distance_melt.loc[:, 'Distance'] = distance_melt.loc[:, 'Distance'] + 1

    fig = plt.figure(figsize=(9, 4), dpi=150)
    plt.title(test_string_2)
    ax = sns.boxplot(x='BodyPart', y='Distance', data=distance_melt,
                     color=".7", fliersize=0, width=0.3, notch=True)
    ax = sns.stripplot(x='BodyPart', y='Distance', data=distance_melt,
                       size=4, jitter=0.25, alpha=.35)
    sns.despine()
    ax.set_yscale("log")
    present_xticks = ax.get_xticks()
    plt.xticks(present_xticks-0.4, rotation=30)
    ax.xaxis.set_ticks_position('none')
    plt.xlabel('')
    plt.ylabel('Distance [micron]')
    plt.ylim([0.25, 1500])

    plt.savefig(fname, dpi=300, bbox_inches='tight')
    
    return distance_2

def plotQuantiles(fname, q_melt_all):
    s = q_melt_all.columns.values.tolist()
    l3 = [x for x in s if x not in ['quantiles', 'BodyPart']]
    q_melt_all_all = pd.melt(q_melt_all, id_vars=['quantiles', 'BodyPart'], var_name='SampleName', value_name='Distance')

    fig = plt.figure(figsize=(6,8), dpi=150)
    temp_data = q_melt_all_all.loc[q_melt_all_all.BodyPart != 'FrameNumber', :]
    ax = sns.stripplot(x='Distance', y='BodyPart', hue='quantiles',
                       dodge=True, data=temp_data, size=5,
                       jitter=0.25, alpha=.25, palette='Set2', zorder=1)
    ax = sns.pointplot(x="Distance", y="BodyPart", hue="quantiles",
                       dodge=0.5, data=temp_data,  join=False,
                       markers="d", ci=None, palette=sns.color_palette("Set2", desat=.9), size=0.5)

    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    sns.despine(bottom=True, left=True)
    ax.xaxis.grid(True)
    plt.xlim([0, 400]);
    ax.yaxis.set_ticks_position('none')
    plt.ylabel('');
    plt.xlabel('Distance [micron]');

    # Improve the legend 
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[:5], labels[:5], title="Quantiles",
              bbox_to_anchor=(1.05, 1), borderaxespad=0., frameon=False)

    plt.savefig(fname, dpi=300, bbox_inches='tight')


In [2]:
## Set path to the directory with test list
# test_grps_list = '../config/test_list' ## small list of two grps
# test_grps_list = '../config/test_groups_list' ## complete list 20180417_7c2

# test_grps_list = '../config/trainingData_20170317_7c1_test' ## complete list 20170317_7c1
# test_grps_list = '../config/trainingData_20180417_7c1_test' ## complete list 20180417_7c1
# test_grps_list = '../config/trainingData_20180417_7c1_Hess_50_test' ## complete list 20180417_7c1_Hess_50
# test_grps_list = '../config/trainingData_20180417_7c1_Hess_50_Test_25_test' ## complete list 20180417_7c1_Hess_50_Test_25
# test_grps_list = '../config/trainingData_20180417_7c1_Hess_test_25_test' ## complete list 20180417_7c1_Hess_25
# test_grps_list = '../config/trainingData_20180417_Individual_test' ## complete list 
# test_grps_list = '../config/trainingData_20180417_7c1_Hess_100_LeftDO_test' ## complete list 
test_grps_list = '../config/trainingData_20180417_7c1_Hess_100_BothDO_test' ## complete list 
# 20170317_7c1

# test_grps_list = '../config/trainingData_20180417_7c0_test' ## complete list 20180417_7c0
# test_grps_list = '../config/trainingData_20170317_7c0_test' ## complete list 20170317_7c0

## ROC Parameters
outlierDistAll = [25, 50, 75, 100];
# vote_thresholds = np.arange(0.0, 30.0, 1);
vote_thresholds = np.arange(0.0, 1.05, 0.05)
detectBodyParts = ['MouthHook', 'LeftMHhook',
                   'RightMHhook', 'LeftDorsalOrgan',
                   'RightDorsalOrgan', 'CenterBolwigOrgan']
rocDatGrpAll = []
it=-1
with open(test_grps_list.strip()) as test_grps:
    n_grps = [i for i in test_grps] 
    for grp in tqdm_notebook(n_grps, total=len(n_grps), unit="grps", desc='Test groups'):
        test_list_file_parent = os.path.split(grp)[0]
        grp_num = str.split(grp, '_grp_')[2].split('_')[0]
        meta_coord_grp = None
        it_grp=-1
        
        with open(grp.strip()) as test_list:
            n_list = [i for i in test_list]
            for test_file in tqdm_notebook(n_list, total=len(n_list), unit="list", desc='Test item', leave=False):
                test_string = str.split(os.path.split(test_file.strip())[1], '_Coordinates')[0]
                
                it += 1
                it_grp += 1
                ## Set path to the directory with tracker metadata and annotation
                test_dir = os.path.join(test_list_file_parent, test_string)
                test_string_2 = str.split(test_string, '_')[2]

                ## Read tracker metadata and annotation
                for fs in os.listdir(test_dir):
                    if 'Metadata' in fs:
                        meta_data = pd.read_csv(os.path.join(test_dir, fs), sep=',', header=0, names=meta_data_header)
                    if 'Coordinates' in fs:
                        coordinates = pd.read_csv(os.path.join(test_dir, fs), sep=',', names=coordinate_header)

                if (meta_data.empty is False):

                    ## Make the metadata frame number start from 0 index
                    meta_data.loc[:, 'FrameNumber'] = meta_data.loc[:, 'FrameNumber'].copy() - 1
                    meta_data.set_index('FrameNumber', inplace=True)

                    ## Make the annotation frame umber start from 0 index
                    coordinates = coordinates.round(0)
                    start_frame = coordinates.loc[0, 'FrameNumber'].copy() - 1
                    coordinates.loc[:, 'FrameNumber'] = coordinates.loc[:, 'FrameNumber'].copy() - start_frame
                    coordinates.set_index('FrameNumber', inplace=True)

                    ## Inner join to take only the frames intersecting the annotation and tracker metadata
                    ## Use when want to see only annotated frames 
                    meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='inner', suffixes=('_T', '_A'))

                    ## Outer join to take only the frames intersecting the annotation and tracker metadata
                    ## Use when want to see all frames 
                #     meta_coord = pd.merge(meta_data, coordinates, on='FrameNumber', how='outer', suffixes=('_T', '_A'))

                    ## Replace NaN values with a high negative number
                    meta_coord.fillna(-2000, inplace=True)

                    ## Calculate distance between annotation and the tracker metadata
                    distance = pd.DataFrame([], index=meta_coord.index.values)
                    distance.index.name = 'FrameNumber'
                    for head in distance_header:
                        temp_x = (meta_coord[head+'_x_T'].values - meta_coord[head+'_x_A'].values)**2
                        temp_y = (meta_coord[head+'_y_T'].values - meta_coord[head+'_y_A'].values)**2
                        distance.loc[:, head+'_dist'] = np.sqrt(temp_x + temp_y)
                    distance[distance > 2716] = np.nan
                    distance = distance*2.75
                    
                    ## Now append the distance table to the main table 'meta_coord'
                    meta_coord = meta_coord.merge(distance, on='FrameNumber')

                    ## Get ROC Curve Data --- Takes time.... uncommented to reduce time
#                     rocDat = {}
#                     rocDat = getROCcurve(meta_coord, detectBodyParts, outlierDistAll, vote_thresholds)
#                     rocFile = os.path.join(test_dir, 'ROC_'+ test_string + '.pkl')
#                     with open(rocFile, 'w') as rf:
#                         pickle.dump(rocDat, rf)
#                     print 'Pickled ROC data for "%s"'%(test_string)
                    
#                     figName = os.path.join(test_dir, 'ROC_'+ test_string + '.png')
#                     plotROC_curve(rocDat, detectBodyParts, outlierDistAll, figName)

#                     figName = os.path.join(test_dir, 'PR_'+ test_string + '.png')
#                     plotPR_curve(rocDat, detectBodyParts, outlierDistAll, figName)
                  
                    ## Plot Distance Box Plot - Individual 
                    figName = os.path.join(test_dir, 'BOX_'+ test_string + '.png')
                    distance_2 = plotDistanceBoxPlot(meta_coord, figName, test_string_2)

                    ## Compute quantile data
                    q = distance_2.quantile([.10, 0.25, 0.5, 0.75, 0.9]).copy()
                    q.reset_index(inplace=True)
                    q.rename(columns={'index': 'quantiles'}, inplace=True)
                    q.loc[:, 'quantiles'] = q.loc[:, 'quantiles'].copy() * 100
                    q = q.iloc[:, :-2].copy()
                    q_melt = pd.melt(q, id_vars='quantiles', var_name='BodyPart', value_name=test_string_2)
                    
                    if it_grp == 0:
                        meta_coord_grp = meta_coord
                    else:
                        meta_coord_grp = pd.concat([meta_coord_grp, meta_coord], axis=0)
                        
                    if it == 0:
                        q_melt_all = q_melt
                        meta_coord_all = meta_coord
                    else:
                        q_melt_all = pd.concat([q_melt_all, q_melt.loc[:, test_string_2]], axis=1)
                        meta_coord_all = pd.concat([meta_coord_all, meta_coord], axis=0)

        ## Plot Distance Box Plot - Group
        figName = os.path.join(test_list_file_parent, 'BOX_GRP_'+ grp_num + '.png')
        distance_2 = plotDistanceBoxPlot(meta_coord_grp, figName, 'GRP_%s'%(grp_num))

        ## Get ROC and PR Curve Data - Group
        rocDatGrp = {}
        rocDatGrp = getROCcurve(meta_coord_grp, detectBodyParts, outlierDistAll, vote_thresholds)
        rocDatGrpAll.append(rocDatGrp)

        figName = os.path.join(test_list_file_parent, 'ROC_Curve_Group.png')
        plotROC_curve(rocDatGrp, detectBodyParts, outlierDistAll, figName)
        
        figName = os.path.join(test_list_file_parent, 'PR_Curve_Group.png')
        plotPR_curve(rocDatGrp, detectBodyParts, outlierDistAll, figName)
        
        figName = os.path.join(test_list_file_parent, 'Recall_Curve_Group.png')
        plotRecall_curve(rocDatGrp, detectBodyParts, outlierDistAll, figName)
        
        plt.close('all')

## Get Quantile Box Plot Data - All
# figName = os.path.join('QuantileDistribution_All.png')
# plotQuantiles(figName, q_melt_all)

## Get ROC Curve Data For all Data -- Not that informative
# rocDatAll = {}
# rocDatAll = getROCcurve(meta_coord_all, detectBodyParts, outlierDistAll, vote_thresholds)

# figName = os.path.join('ROC_Curve.png')
# plotROC_curve(rocDatAll, detectBodyParts, outlierDistAll, figName)

# figName = os.path.join('PR_Curve.png')
# plotPR_curve(rocDatAll, detectBodyParts, outlierDistAll, figName)

rocFile = 'ROC_' + str.split(os.path.split(os.path.split(test_list_file_parent)[0])[1], 'trainingData_')[1] + '.pkl'
with open(rocFile, 'w') as rf:
    pickle.dump(rocDatGrpAll, rf)